In [2]:
import pandas as pd
import sqlite3
import os

In [4]:
os.makedirs("raw", exist_ok=True)
os.makedirs("processed", exist_ok=True)
os.makedirs("output", exist_ok=True)

In [8]:
df = pd.read_csv(r"D:\Downloads\customer_churn_dataset-testing-master.csv")
print(df.head())
print(df.shape)

   CustomerID  Age  Gender  Tenure  Usage Frequency  Support Calls  \
0           1   22  Female      25               14              4   
1           2   41  Female      28               28              7   
2           3   47    Male      27               10              2   
3           4   35    Male       9               12              5   
4           5   53  Female      58               24              9   

   Payment Delay Subscription Type Contract Length  Total Spend  \
0             27             Basic         Monthly          598   
1             13          Standard         Monthly          584   
2             29           Premium          Annual          757   
3             17           Premium       Quarterly          232   
4              2          Standard          Annual          533   

   Last Interaction  Churn  
0                 9      1  
1                20      0  
2                21      0  
3                18      0  
4                18      0  
(6

In [10]:
# Remove duplicates
df = df.drop_duplicates()

# Handle missing values
df = df.dropna()

# Standardize column names
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [12]:
if "profit" in df.columns and "sales" in df.columns:
    df["profit_margin"] = df["profit"] / df["sales"]
else:
    df["profit_margin"] = 0.2  # dummy if not available

In [16]:
print(df.columns)

Index(['customerid', 'age', 'gender', 'tenure', 'usage_frequency',
       'support_calls', 'payment_delay', 'subscription_type',
       'contract_length', 'total_spend', 'last_interaction', 'churn',
       'profit_margin'],
      dtype='object')


In [18]:
customers = df[[
    "customerid",
    "age",
    "gender",
    "subscription_type",
    "contract_length"
]].drop_duplicates()

In [20]:
usage = df[[
    "customerid",
    "tenure",
    "usage_frequency",
    "support_calls",
    "payment_delay",
    "total_spend"
]]

In [22]:
churn = df[[
    "customerid",
    "churn",
    "last_interaction"
]]

In [24]:
customers.to_csv("processed/customers.csv", index=False)
usage.to_csv("processed/usage.csv", index=False)
churn.to_csv("processed/churn.csv", index=False)

In [26]:
import sqlite3

conn = sqlite3.connect("output/database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
usage.to_sql("usage", conn, if_exists="replace", index=False)
churn.to_sql("churn", conn, if_exists="replace", index=False)

conn.close()

In [28]:
print("Original rows:", len(df))
print("Customers:", len(customers))
print("Usage:", len(usage))
print("Churn:", len(churn))

Original rows: 64374
Customers: 64374
Usage: 64374
Churn: 64374


ETL mini data pipeline built using Python and pandas to extract, transform, and load customer churn data into structured CSV files and SQLite database.